In [1]:
import tensorflow as tf
from tensorflow.python.keras import models as m
from tensorflow.python.keras import layers as l

import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.metrics import confusion_matrix, fbeta_score
from scikitplot.metrics import plot_roc, plot_confusion_matrix

import seaborn as sn
import pandas as pd

from ROOT import TFile, TTree, TChain
from ROOT import TH1D, TH2D, TCanvas
from ROOT import gDirectory, TGraph, TMultiGraph
import ROOT as rt
import os
import sys
sys.path.insert(0, os.path.abspath('/home/arsahasransu/Documents/SoftDisplacedLeptons/Classifier/'))

print("All classes initialized successfully!!!")


import plotBeautifier as pB
pB.trial_func("AR")

Welcome to JupyROOT 6.18/04
All classes initialized successfully!!!
Trial works fine. Hello,AR


In [3]:
bkgchain = TChain("varTree")
bkgchain.Add("../background.root")
bkgfull = bkgchain.AsMatrix()
    
# Load the input data scaler
scaler = joblib.load("../Classifier/scaler.save")

# Load the model
loaded_model = m.load_model("../Classifier/simplePer.h5")

bkgscaled = scaler.transform(bkgfull)
bkgpredict = loaded_model.predict(bkgscaled)

bkgsigprob = np.array(bkgpredict)[:,0]

In [19]:
discut = 0.9

sigchain = TChain("varTree")
sigchain.Add("../BP_200_40_20.root")

sigfull = sigchain.AsMatrix()
sigfull = np.transpose(sigfull)

evntweight = sigfull[0]
d0el = sigfull[1]
domu = sigfull[2]
sigfull = sigfull[3:12]
sigfull = np.transpose(sigfull)

sigscaled = scaler.transform(sigfull)
sigpredict = loaded_model.predict(sigscaled)
sigsigprob = np.array(sigpredict)[:,0]
sigclass = np.array(sigsigprob>=discut, dtype=int)
evntcnt = evntweight*sigclass

print(evntweight)
print(sigsigprob)
print(sigclass)
print(evntcnt.sum())

[0.61350241 0.74876499 0.70520688 ... 0.87135611 0.72785469 0.7323452 ]
[0.99796224 0.77449995 0.7618678  ... 0.90468025 0.9929645  0.9992895 ]
[1 0 0 ... 1 1 1]
35632.23298511684
